<a href="https://colab.research.google.com/github/richa-giri/Digitide-Work/blob/main/RefundPolicy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install langchain langchain-community faiss-cpu sentence-transformers transformers accelerate


In [ ]:
company_policy = """
Company Policy Document

Refund Policy:
We offer a full refund within 30 days of purchase if the product is unused and in its original packaging.
Refunds will be processed within 7 business days after approval.
Shipping fees are non-refundable.

Return Exceptions:
Digital goods are not eligible for refunds unless explicitly stated.
"""

with open("policy.txt", "w", encoding="utf-8") as f:
    f.write(company_policy)


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

loader = TextLoader("policy.txt", encoding="utf-8")
docs = loader.load()

splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = FAISS.from_documents(chunks, embeddings)


In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline

pipe = pipeline(
    task="text2text-generation",
    model="google/flan-t5-small",
    device=-1,  # CPU; use device=0 for GPU
    max_length=256,
)

hf_llm = HuggingFacePipeline(pipeline=pipe)

qa = RetrievalQA.from_chain_type(
    llm=hf_llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)


In [ ]:
query = "What is the refund policy?"
answer = qa.run(query)

print("Question:", query)
print("Answer:\n", answer)
